In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [748 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ P

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-16 17:21:11--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.59MB/s    in 0.2s    

2022-05-16 17:21:12 (5.59 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataAppliances").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [5]:
rows=df.count()
rows

96901

In [6]:
# Drop duplicates and incomplete rows 
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

96901
96888
96888


In [7]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
customers_df = df.groupby("customer_id").agg({"customer_id":"count"})\

customers_df.orderBy("count(customer_id)").show(truncate=False)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|14664611   |1                 |
|21458971   |1                 |
|49857015   |1                 |
|31196655   |1                 |
|47319296   |1                 |
|13246430   |1                 |
|33085426   |1                 |
|52231496   |1                 |
|36173316   |1                 |
|37856551   |1                 |
|44309150   |1                 |
|1263434    |1                 |
|49355710   |1                 |
|23952664   |1                 |
|21737233   |1                 |
|45155320   |1                 |
|52628848   |1                 |
|19909263   |1                 |
|21782950   |1                 |
|27917440   |1                 |
+-----------+------------------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import desc
customers2_df = customers_df.withColumnRenamed("customer_id", "customer_id")\
                                     .withColumnRenamed("count(customer_id)", "customer_count")
customers2_df.orderBy(desc("customer_count")).show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   32962808|            16|
|   15489471|            15|
|    5695524|            15|
|   30396577|            14|
|   34553562|            13|
|   51042275|            12|
|   32230187|            12|
|   29748841|            12|
|   51139148|            12|
|   12914327|            11|
|   40382895|            11|
|   13649055|            11|
|   30544694|            11|
|   48159861|            11|
|   20862753|            11|
|   49444310|             9|
|   45671123|             9|
|   14384334|             9|
|   24715941|             9|
|   53074513|             9|
+-----------+--------------+
only showing top 20 rows



In [10]:
customers2_df.count()

91299

In [11]:
customers2_df.dtypes

[('customer_id', 'int'), ('customer_count', 'bigint')]

In [12]:
# Configuration for RDS instance
mode="overwrite"
jdbc_url = "jdbc:postgresql://****.rds.amazonaws.com:5432/bd_db"
config = {"user":"postgres",
          "password": "****",
          "driver":"org.postgresql.Driver"}

In [13]:
customers2_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [14]:
product_df = df.select(["product_id","product_title"])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00E66SE2U|Part # 3390719 - ...|
|B00SNKU7VQ|Whirlpool W102953...|
|B00NN166DS|GE GFE28HSHSS 27....|
|B00AW04ZMM|Panda Small Compa...|
|B00GXHRC6E|LG H/E Super Capa...|
|B00J8A9S4C|Bosch NGM8655UC 8...|
|B00413662I|Amana 1.5 cu. ft....|
|B00FN898HU|GOLDEN VANTAGE 30...|
|B005KT4LK6|Whirlpool WTW8800...|
|B00N0GDA9O|Fisher Paykel RF1...|
|B00DYJNNWQ|Luma Comfort IM20...|
|B005GM942C|Panda Small Mini ...|
|B005DUT1NE|Maytag MVWB950YW ...|
|B00B2NPKRM|Dishwaser Stainle...|
|B0058ENV2W|Midea White/Black...|
|B007OWOK7Q|DISPOSABLE FOIL B...|
|B00KD5CCZ8|NewAir AI-100R 28...|
|B00JG8B42K|OnePurify RFC0800...|
|B00BAXC2W0|NY Cake Mold Sili...|
|B0039V5HZK|LG DLE5001W7.3 Cu...|
+----------+--------------------+
only showing top 20 rows



In [15]:
# Drop duplicates and incomplete rows 
print(product_df.count())
product_df = product_df.dropna()
print(product_df.count())
product_df = product_df.dropDuplicates()
print(product_df.count())

96888
96888
11694


In [16]:
product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [17]:
# Select the column headers in the review_id_table schema
from pyspark.sql.functions import to_date

to_date("review_date", 'yyyy-MM-dd').alias()

review_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R354ZG794Q5KJQ|   26514959|B00E66SE2U|     199871830| 2015-08-30|
| RNZ36HB1YSQ6G|   48250444|B00SNKU7VQ|     197365398| 2015-08-29|
| R2MHXTICGUDVE|    3420426|B00NN166DS|     546911168| 2015-08-29|
|R172DDSWTL0P9K|     539114|B00AW04ZMM|     962476921| 2015-08-22|
|R1SUJJFCOKC7HI|   13016134|B00GXHRC6E|     348942012| 2015-08-22|
| R1EGFDXPYGROC|   11324103|B00J8A9S4C|     772573004| 2015-08-22|
|R3LSXWYAIE6WBG|   17539277|B00413662I|     801135043| 2015-08-22|
| REIN0JDLBLFHA|   17299942|B00FN898HU|     462369678| 2015-08-22|
|R2EZ2UXVLUANE8|   22355805|B005KT4LK6|     236627965| 2015-08-20|
| R9XCQJZA5BKO6|   45332207|B00N0GDA9O|     286297774| 2015-08-20|
|R3A91XAOIHUNTW|   15067373|B00DYJNNWQ|     779409171| 2015-08-18|
|R144T3UALNFF1L|   14217524|B005GM942C|     788261054| 2015-08

In [18]:
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)